## Umgebung vorbereiten

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/edadunashvili/ThePrax.git

In [ ]:
cd ThePrax

In [ ]:
!pwd

In [ ]:
!pip install pyprind

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Testdatencorpushersteller

 !!! Ein Typ bestimmen !!!

In [ ]:
typNum='300'
episode_string_train = "a300_string_train.csv"
episode_roh_train = "a300_roh_train.csv"

Vorhandener gleichnamiger Korpus wird gelöscht 

In [ ]:
import os
if os.path.exists(episode_string_train):
    os.remove(episode_string_train)
else:
    print("Diese Datei existiert nicht")

Im Ordner "Trainingsdaten" nach den entsprechenden Textdateien suchen und in einer rohe Datei zusammentragen

In [ ]:
import glob

def word_to_lex(word):
    ret=(word) 
    return ret

def write_back(words):
    with open(episode_roh_train,"a", encoding='utf-8') as output:
        for word in words:
            #print(word)
            as_lex = word_to_lex(word[0])
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)

def clean(line):
    line = line.replace("\n"," ").strip().lower()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe").replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","").replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a").replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ").replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ").replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','').replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret

with open(episode_roh_train, "w", encoding='utf-8') as output:
    output.write ("quelle,episode,index_string,index_binar\n")
pairs = []


for file in glob.glob("Trainingsdaten/*.txt"):
    
    if typNum in file:
        with open(file, 'r', encoding='utf-8') as episode:
            for line in episode.readlines():
                clean_words = clean(line)
                pairs = pairs + clean_words
write_back(pairs)

Rohdatei endgültig überarbeiten und in eine CSV Datei speichern

In [ ]:
fin = open(episode_roh_train,'r', encoding ='utf-8')
fout = open(episode_string_train, "wt", encoding ='utf-8')
for kfz in fin:
    fout.write(kfz.replace(', "["',"").replace('"[', "").replace(']"',"").replace("', '", "','").replace(" '", "'"))                      
fin.close()
fout.close()

Liste von einmaligen Episoden aus der CSV Datei zeigen

In [ ]:
import pandas as pd
df = pd.read_csv(episode_string_train, encoding='utf-8')
from collections import Counter
indexliste=Counter(df.index_string)
print(*indexliste, sep='\n')

## Auswerter

Elementenstruktur definieren: vorherige Episode - gesuchte Episode - folgende Episode

!!! Datei definieren !!!

In [ ]:
import pandas as pd
df = pd.read_csv('a300_string_train.csv', encoding='utf-8')
from collections import Counter

def ep_format(ep_full):
    return (ep_full.split('_'))[0]

def quellenvergleich (df, i1, i2):
#     print(df.quelle[i1],df.quelle[i2])
    return df.quelle[i1]==df.quelle[i2]

def ast(gesep, df):
    ep_tree = {}
    a_liste = []
    z_liste = []  
    df_len = len(df.index_string)
    for i, ep in enumerate(df.index_string):
        if gesep == ep:
             #print(i, ep_full)
            if (i > 0)&(quellenvergleich(df, i, i-1)):
                a = df.index_string[i-1]
            else:
                a = 'eAnfang_a_'
            if (i < df_len - 1):
                if not (quellenvergleich(df, i, i+1)):
                    z = 'eEnde_a_'
                else:        
                    z = df.index_string[i+1]
            else:
                z = 'eEnde_a_'
            a_liste.append(a)
            z_liste.append(z)
    return {gesep: [Counter(a_liste), Counter(z_liste)]}

Alle im Korpus vorhandene Episoden extrahieren

In [ ]:
def alle_aeste(gesep, df):
    episoden_baeume = {}
    
    ep_list = []
    for ep_full in df.index_string:  
        
        ep = ep_format(ep_full)      
        if gesep == ep:          
            
            ep_list.append(ep_full)
    for ep in set(ep_list):
        episoden_baeume.update(ast(ep,df))
    return episoden_baeume

Episoden die gesuchte Episode umlegen und in die Liste in die richrige Reihenfolge darstellen

In [ ]:
def key_val_printer(d):
    for k, v in d.items():
        print(v, k, sep=':')
    #print()
        
def baum_printer(baeume: dict, baum: str):
    key_val_printer (baeume[baum][0])
    
    
    print(80*'-')
    print(sum(baeume[baum][0].values()), baum)
    print(80*'-')
    key_val_printer(baeume[baum][1]) 
    
            
def wald_printer(wald: dict):
    for baum in sorted(wald.keys()):
        baum_printer (wald, baum)
        print(80*'=')

!!! Die gesuchte Episode bestimmen !!!

In [ ]:
synop = alle_aeste("'e300", df)
wald_printer(synop)

## Modell erstellen und trainieren

In [ ]:
import keras
import glob
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pyprind
import pandas as pd
import random
import re
import string
from collections import Counter
from nltk.corpus import stopwords
from keras import backend as K
from keras import layers
from keras import losses
from keras import models
from keras import metrics
from keras import optimizers
from keras.preprocessing import sequence

!!! Variable anpassen !!!

In [ ]:
ziel_episode = 'e300_e_vorfeld_des_kampfes_bis_zum_sieg'
mini_frequenz = 2
k_fach = 5
layD = 96
episode_string_train = 'a300_string_train.csv' 
episode_binar_train = 'a300_binar_train.csv'
episode_string_mess = 'gesamt_string_mess.csv'
episode_binar_mess = 'gesamt_binar_mess.csv'

Aufbereitung der Trainingsdaten
Rohdaten laden und und den Episodenbestand betrachten

In [ ]:
fin = open(episode_string_train,'r', encoding='utf-8') 
fout = open(episode_binar_train, "wt", encoding='utf-8')
for efz in fin:
    fout.write(efz.replace("'",""))  
fin.close()
fout.close()

In [ ]:
df = pd.read_csv(episode_binar_train, encoding='utf-8')
indexliste=Counter(df.index_string)
print(*indexliste, sep='\n')

Die gesuchte Episode etikettieren und den Rohdaten in Trainingsdaten umwandeln

In [ ]:
for i,e in enumerate(df.index_string):
    if e == ziel_episode:
        df.index_binar[i]='1'
    else: 
        df.index_binar[i]='0'

In [ ]:
df.to_csv(episode_binar_train, encoding='utf-8', index=False)

Trainngsdaten aufbereiten
4.1. Das Vorkommen jedes Wortes zählen

In [ ]:
nltk.download('stopwords')
w = stopwords.words('german')
counts = Counter()
pbar = pyprind.ProgBar(len(df['episode']),
                       title='Vorkommen der Wörter zählen')
for i, episode in enumerate(df['episode']): 
        pbar.update()
        counts.update(episode.split())
new_counts = {}
for k, v in counts.items():
    if v > mini_frequenz and k not in w:
        new_counts[k] = v
counts = Counter(new_counts)

Zuordnung erzeugen und den verschiedenen Wörtern eindeutige Zahlen zuordnen

In [ ]:
word_counts = sorted(counts, key=counts.get, reverse=True)
word_to_int = {word: ii for ii, word in enumerate(word_counts,0)}
mapped_episoden = []
pbar = pyprind.ProgBar(len(df['episode']),
                       title='Episoden Zahlen zuordnen')
for episode in df['episode']:
    mapped_episoden.append([word_to_int.get(word) for word in episode.split()])
    pbar.update()
mapped_episoden = [list(filter(None, el)) for el in mapped_episoden]

Traiingsndaten und Trainingslabels bestimmen

In [ ]:
train_data = mapped_episoden[0:] 
train_labels = df.loc[0:, 'index_binar'].values
print("sequences =",max([max(sequences) for sequences in mapped_episoden if len(sequences)>0])," ",
      "train_data =", len (train_data))

Trainingsdaten und -Labels vektorisieren

In [ ]:
sequences_laenge=max([max(sequences) for sequences in mapped_episoden if len(sequences)>0])
def vectorize_sequences(sequences, dimension=sequences_laenge+1): 
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
x_train = vectorize_sequences(train_data) 
y_train = np.asarray(train_labels).astype('float32')

In [ ]:
#for i, element in enumerate(x_train[63]):
    #print(i,'-', element)

Neuronales Netz erzeugen

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(layD, activation='tanh',
                           input_shape=(sequences_laenge+1,)))
    #model.add(layers.Dense(layD, activation='relu'))
    model.add(layers.Dense(layD, activation='tanh')) 
    model.add(layers.Dense((1), activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['binary_accuracy'])
    return model

Neuronales Netz trainieren

In [ ]:
k = k_fach
num_val_samples = len(x_train) // k 
num_epochs = 4
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_x_train = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_y_train = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_x_train, partial_y_train,
              epochs=num_epochs, batch_size = 8, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_binary_accuracy = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_binary_accuracy)

In [ ]:
all_scores

In [ ]:
np.mean(all_scores)

In [ ]:
num_epochs = 8
all_binary_accuracy_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_x_train = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_y_train = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_x_train, partial_y_train,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size = 8, verbose=0)
    binary_accuracy_history = history.history['val_binary_accuracy']
    all_binary_accuracy_histories.append(binary_accuracy_history)

In [ ]:
average_binary_accuracy_history = [
    np.mean([x[i] for x in all_binary_accuracy_histories]) for i in range(num_epochs)]
for i, element in enumerate(average_binary_accuracy_history):
    print(i,'-', element)

## Messdaten erstellen

!!! Ein Typ bestimmen !!!

In [ ]:
typNum =''
episode_string_train = "gesamt_string_mess.csv"
episode_roh_train = "gesamt_roh_mess.csv"

Alte Datei mit gleichem Namen löschen

In [ ]:
import os
if os.path.exists(episode_string_train):
    os.remove(episode_string_train)
else:
    print("Diese Datei existiert nicht")

Nach den entsprechenden Textdateien im Ordnet suchen und in einer rohe Datei zusammentragen

In [ ]:
import glob

def word_to_lex(word):
    ret=(word) 
    return ret

def write_back(words):
    with open(episode_roh_train,"a", encoding='utf-8') as output:
        for word in words:
            #print(word)
            as_lex = word_to_lex(word[0])
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)

def clean(line):
    line = line.replace("\n"," ").strip().lower()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe").replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","").replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a").replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ").replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ").replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','').replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret

with open(episode_roh_train, "w", encoding='utf-8') as output:
    output.write ("quelle,episode,index_string,index_binar\n")
pairs = []

for file in glob.glob("Messdaten/*.txt"):
    if typNum in file:
        with open(file, 'r', encoding='utf-8') as episode:
            for line in episode.readlines():
                clean_words = clean(line)
                pairs = pairs + clean_words
write_back(pairs)

Rohdatei endgültig überarbeiten und in eine CSV Datei speichern

In [ ]:
fin = open(episode_roh_train,'r', encoding ='utf-8')
fout = open(episode_string_train, "wt", encoding ='utf-8')
for kfz in fin:
    fout.write(kfz.replace(', "["',"").replace('"[', "").replace(']"',"").replace("', '", "','").replace(" '", "'"))                      
fin.close()
fout.close()

Liste von einmaligen Episoden erstellen in der CSV Datei

In [ ]:
import pandas as pd
df = pd.read_csv(episode_string_train, encoding='utf-8')
from collections import Counter
indexliste=Counter(df.index_string)
print(*indexliste, sep='\n')

Messdaten laden und aufbereiten

In [ ]:
fin = open(episode_string_mess,'r', encoding='utf-8') 
fout = open(episode_binar_mess, "wt", encoding='utf-8')
for efz in fin:
    fout.write(efz.replace("'",""))
                                 
fin.close()
fout.close()

In [ ]:
df = pd.read_csv(episode_binar_mess, encoding='utf-8')
indexliste=Counter(df.index_string)
print(indexliste, sep='\n')

In [ ]:
for i,e in enumerate(df.index_string):
    if e:
        df.index_binar[i]='0'
    else: 
        df.index_binar[i]='0'

In [ ]:
df.to_csv(episode_binar_mess, encoding='utf-8', index=False)

Vorhandene Zuordnung laden und den verschiedenen Wörtern eindeutige Zahlen zuordnen

In [ ]:
mapped_episoden_2 = []
pbar = pyprind.ProgBar(len(df['episode']),
                       title='Episoden Zahlen zuordnen')
for episode in df['episode']:
    mapped_episoden_2.append([word_to_int.get(word) for word in episode.split()])
    pbar.update()

mapped_episoden_2 = [list(filter(None, el)) for el in mapped_episoden_2]


raiingsndaten und Trainingslabels bestimmen

In [ ]:
mess_data = mapped_episoden_2[0:] 
mess_labels = df.loc[0:, 'index_binar'].values
print("sequences =",max([max(sequences) for sequences in mapped_episoden if len(sequences)>0])," ", 
      "mess_data =", len (mess_data))

Messdaten vektorisieren

In [ ]:
def vectorize_sequences(sequences, dimension=sequences_laenge+1): 
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
x_mess = vectorize_sequences(mess_data)
y_mess = np.asarray(mess_labels).astype('float32')

## Messdaten analysieren

In [ ]:
plt.plot(range(1, len(average_binary_accuracy_history) + 1), average_binary_accuracy_history)
plt.xlabel('Epochs')
plt.ylabel('Validation acc')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_binary_accuracy_history = smooth_curve(average_binary_accuracy_history[3:])

plt.plot(range(1, len(smooth_binary_accuracy_history) + 1), smooth_binary_accuracy_history)
plt.xlabel('Epochs')
plt.ylabel('Validation acc')
plt.show()

Modell ausführen

In [ ]:
# Get a fresh, compiled model.
model = build_model()
# Train it on the entirety of the data.
model.fit(x_train, y_train,
          epochs = 2, batch_size = 1)
mess_mse_score, mess_binary_accuracy_score = model.evaluate(x_mess, y_mess)

In [ ]:
mess_binary_accuracy_score, mess_mse_score

Voraussagen

In [ ]:
predictions=model.predict(x_mess)
#for i in range(len(predictions)):
    #print(model.predict(x_mess))

In [ ]:
print('Maximaler Wert:', max(max(model.predict(x_mess))))

Voraussgen in den aussortierten Datensätzen

In [ ]:
mod_pred = model.predict(x_mess)
cutoff_value = 0.1
for i in range(len(y_mess)):
    if mod_pred[i,0]>cutoff_value:
        if ((i==0)|(mod_pred[i-1,0]>cutoff_value))|(mod_pred[i+0]>cutoff_value):
            #if df.index_string ==0:
                print (i+0,"-",
                  #i+2,"-", 
                  #y_test[i],"-", 
                  mod_pred[i],"-",
                  df.quelle[i+0],"-", 
                  df.episode[i+0],"-",
                  df.index_string[i+0],'\n')

Voraussage: gewünschte Datensetze abrufen

In [ ]:
for i in range(len(x_mess)):
    if i>3 and i<14:
        print(i+0,"-",
              model.predict(x_mess)[i],"-",
              df.quelle[i+0],"-",
              df.episode[i+0],"-",
              df.index_string[i+0],'\n')